In [13]:
import pandas as pd
import numpy as np
import itertools
from collections import Counter
from datetime import datetime
import re
import ast

# from pandas_profiling import ProfileReport

import my_functions
import importlib
importlib.reload(my_functions)

import matplotlib.pyplot as plt
%matplotlib inline 
import seaborn as sns
from scipy import stats


In [14]:
RANDOM_SEED = 123

In [15]:
input_folder = 'input//'
print(os.listdir("input"))

['main_task.csv', 'kaggle_task.csv', 'sample_submission.csv']


In [16]:
data = pd.read_csv(input_folder + 'main_task.csv')
data_kaggle = pd.read_csv(input_folder + 'kaggle_task.csv')
sample_submission = pd.read_csv(input_folder + '/sample_submission.csv')

# дря корректной обработки признаков объединяем трейн и тест в один датасет
data['sample'] = 1 # помечаем где у нас трейн
data_kaggle['sample'] = 0 # помечаем где у нас тест
data_kaggle['Rating'] = 0 # в тесте у нас нет значения Rating, мы его должны предсказать, по этому пока просто заполняем нулями

data = data_kaggle.append(data, sort=False).reset_index(drop=True) # объединяем

In [17]:
preproc_data = my_functions.prepoc_data(data)


In [30]:
# Загружаем специальный инструмент для разбивки:
from sklearn.model_selection import train_test_split
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели
from sklearn.linear_model import LinearRegression

# Теперь выделим тестовую часть
train_data = preproc_data.query('sample == 1').drop(['sample'], axis=1)
test_data = preproc_data.query('sample == 0').drop(['sample'], axis=1)

y = train_data.rating.values            # наш таргет
X = train_data.drop(['rating'], axis=1)

In [27]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)

In [28]:
model = RandomForestRegressor(n_estimators=100, verbose=1, n_jobs=-1, random_state=RANDOM_SEED)

In [29]:
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   13.6s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:   30.3s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.1s finished


In [31]:
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

MAE: 0.20967874999999997


In [23]:
test_data = test_data.drop(['rating'], axis=1)

In [24]:
# predict_submission = model.predict(test_data)

In [25]:
# sample_submission['Rating'] = predict_submission
# sample_submission.to_csv('submission.csv', index=False)
# sample_submission.head(10)